## Query Processing Module

#### Query Input

In [2]:
import nltk
# nltk.download('stopwords')
from pyvis.network import Network
import networkx as nx
import pickle
import bs4

%matplotlib inline

DATA="../data/"

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from os import listdir
from os.path import isfile, join
mypath = "text_misc_clean/"
misc_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "text_casefiles/"
case_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]


1.15.0


In [4]:
from nltk.metrics import edit_distance
#Definitions dictionary
import os, json
import pickle
defdict = {}
path_to_json = DATA + 'Definitions/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
for i in json_files:
    with open(DATA + 'Definitions/'+i) as json_file:
        data = json.load(json_file)
    defdict.update(data)
definitions = list(defdict.keys())
defvalues = list(defdict.values())

docs = ["Issue and Listing of Non Convertible Redeemable Preference Shares", "Investment Advisers", "Depositories and Participants", "Mutual Funds", "Employees Service", "Substantial Acquisition of Shares and Takeovers", "Appointment of Administrator and Procedure for Refunding to the Investors", "Prohibition of Fraudulent and Unfair Trade Practices relating to Securities Market", "Know Your Client Regulations", "Prohibition of Insider Trading", "Merchant Bankers", "Issue and Listing  of Securities Debt Instruments and Security Receipts", "Delisting of Equity Shares","Issue of Capital And Disclosure Requirements2", "Foreign Venture Capital Investor", "Procedure for Board Meetings", "Custodian", "Ombudsman", "Investor Protection and Education Fund", "Foreign Portfolio Investors", "Issue of Sweat Equity", "Collective Investment Scheme", "Portfolio Managers", "Research Analysts", "Procedure for Search and Seizure", "Issue of Capital And Disclosure Requirements", "Share Based Employee Benefits", "Debenture Trustees", "Alternative Investment Funds", "Stock Exchanges and Clearing Corporations", "Self Regulatory Organisations", "Settlement Proceedings", "Issues and Listing of Muncipal Debt Securities", "Buy Back Of Securities2","Issue and Listing of Debt Securities", "Infrastructure Investment Trusts", "Stock Brokers", "Listing Obligations and Disclosure Requirements", "Registrars to an Issue and Share Transfer Agents", "Real Estate Investment Trusts", "Intermediaries", "Certification of Associated Persons in the Securities Markets", "Credit Rating Agencies", "Regulatory Fee on Stock Exchanges", "Underwriters", "Buy Back Of Securities", "Bankers to an Issue", "Central Database of Market Participants"]

#Regulations
finalclean48 = []
with open(DATA + 'cleanedregulations48.pkl','rb') as f:
    finalclean48 = pickle.load(f)    
docregs = finalclean48

#Topics documentwise
finalcleantopics48 = []
with open(DATA + 'cleanedregtopics48.pkl','rb') as f:
    finalcleantopics48 = pickle.load(f)    
finaltopics = finalcleantopics48

#vocab definitions
mainvocab = []
with open(DATA + 'mainvocab.pkl','rb') as f:
    mainvocab = pickle.load(f) 

vocabdef = []
with open(DATA + 'vocabdef.pkl','rb') as f:
    vocabdef = pickle.load(f) 

#Additional Documents

with open(DATA + 'concept_filenames.pkl','rb') as f:
    cfile = pickle.load(f) 
with open(DATA + 'concept_sentences.pkl','rb') as f:
    csent = pickle.load(f) 
with open(DATA + 'concept_text.pkl','rb') as f:
    ctext = pickle.load(f) 

with open(DATA + 'informal_filenames.pkl','rb') as f:
    ifile = pickle.load(f) 
with open(DATA + 'informal_sentences.pkl','rb') as f:
    isent = pickle.load(f) 
with open(DATA + 'informal_text.pkl','rb') as f:
    itext = pickle.load(f) 
    
with open(DATA + 'miscvocab.pkl', 'rb') as f:
    miscvocab = pickle.load(f)
    
#Legal Case files
with open(DATA + 'case_filenames.pkl','rb') as f:
    lfile = pickle.load(f) 
with open(DATA + 'case_sentences.pkl','rb') as f:
    lsent = pickle.load(f) 
with open(DATA + 'case_text.pkl','rb') as f:
    ltext = pickle.load(f) 
    
with open(DATA + 'casevocab.pkl', 'rb') as f:
    casevocab = pickle.load(f)

import spacy
nlp = spacy.load('en_core_web_sm')

from nltk.corpus import wordnet

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import math
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
STOPWORDS = set(
    list(stop_words.ENGLISH_STOP_WORDS) +\
    stopwords.words('english') +\
    ['mm', 'section', 'subsection', 'schedule', '-PRON-', 'chapter', 'regulation', 'repealed', 'thereto','unpublishe', 'thereunder','guideline', 'reference','onus','make','Page','Securities','Exchange','India'])

from cdqa.pipeline import QAPipeline

from cdqa.utils.download import download_model
download_model(model='bert-squad_1.1', dir='./models')

with open(DATA + 'glossary.json') as f:
    glossary = json.load(f)

/home/spawnfire/anaconda3/envs/testingEnvironment/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)



bert_qa.joblib already downloaded


In [26]:
# type(lsent[0])
# print(in)

SyntaxError: invalid syntax (<ipython-input-26-6588326fbb50>, line 2)

In [99]:
def queryvocab(query):
    query1 = ""
    qvocab=[]
    for i in query.split():
        if(i not in stopwords.words()):
            if(i != 'What' and i != 'what' and i!='How'):
                qvocab.append(i)
                query1 = query1 + " " + i
    query = query1
    return qvocab, query

In [100]:
def regextract(docinput, docs, docregs, mainvocab, vocabdef, query, glossary):
    
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    v = []
    
    if(docinput == 'All'):
        rdocs = []
        for i in glossary.keys():
            if(i in query.lower()):
                dval = glossary[i]
                for j in dval:
                    if(j not in rdocs):
                        rdocs.append(j)
        if(len(rdocs)>0):
            docinput = []
            for i in rdocs:
                docinput.append(docs[i])
        else:
            docinput = ['All']
            
    else:
        d = [docinput]
        docinput = d
    
    for dval in docinput:
    
        if(dval != 'All'):
            ind = docs.index(dval)

            rules = docregs[ind]
        
            t=-1
            vocab0 = []
            for i in rules:
                t=t+1
                vocab0.append(i.split())
            
            for i in vocab0:
                for j in i:
                    if(len(j)>2):
                        for m in range(len(symbols)):
                            j = np.char.replace(j, symbols[m], '')
                            j = np.char.replace(j, ",", "")
                            j = np.array2string(j)
                            j = j.replace("'","")
                        if(j not in v):
                            v.append(j)
        else:
            rules = []
            for i in docregs:
                for j in i:
                    rules.append(j)
            vocab0 = mainvocab
        
            for j in vocab0:
                if(len(j)>2):
                    for m in range(len(symbols)):
                        j = np.char.replace(j, symbols[m], '')
                        j = np.char.replace(j, ",", "")
                        j = np.array2string(j)
                        j = j.replace("'","")
                    if(j not in v):
                        v.append(j)
    
    if('All' not in docinput):
        vdef = []
        for i in v:
            if(i in mainvocab and len(i)>2):
                vindex = mainvocab.index(i)
                vdef.append(vocabdef[vindex])
    else:
        vdef = vocabdef
        
    
    return rules, v, vdef, docinput

In [101]:
def querypreprocess(query, qvocab, definitions, finaltopics):
    if('regulations' in qvocab):
        qvocab.remove('regulations')
    if('rules' in qvocab):
        qvocab.remove('rules')
    if('rule' in qvocab):
        qvocab.remove('rule')
    if('chapter' in qvocab):
        qvocab.remove('chapter')
    if('section' in qvocab):
        qvocab.remove('section')
    if('sub' in qvocab):
        qvocab.remove('sub')
    if('SEBI' in qvocab):
        qvocab.remove('SEBI')
    if('means' in qvocab):
        qvocab.remove('means')
    if('shall' in qvocab):
        qvocab.remove('shall')
    if('Securities' in qvocab):
        qvocab.remove('Securities')
    if('Exchange' in qvocab):
        qvocab.remove('Exchange')
    if('pertaining' in qvocab):
        qvocab.remove('pertaining')
    if('India' in qvocab):
        qvocab.remove('India')
        
    query = ""
    for i in qvocab:
        query = query + " " + i
        
    importantwords = []
    for i in qvocab:
        flag = 0
        for j in definitions:
            if(i in j):
                flag = 1
                break
        if(flag == 1):
            importantwords.append(i)

    expansionwords = []
    for i in qvocab:
        flag=0
        for j in finaltopics:
            for k in j:
                if(i == k):
                    flag=1
        if(flag==1):
            if(i not in importantwords):
                importantwords.append(i)
        if(flag==0):
            for j0 in definitions:
                if(i in j0):
                    expansionwords.append(j0)
                    break
    
    qnew = query
    for j in expansionwords:
        k=0
        for i in definitions:
            if(i==j):
                s = i
                s = s + defvalues[k]
                qnew = qnew + ' ' + s
                break
            k=k+1
        else:
            qnew = qnew + ' ' + j
    query = qnew
    
    sent = nlp(query)
    t=0
    for token in sent:
        if(str(token) in importantwords):
            if(t!=0):
                if(str(sent[t-1]) not in importantwords):
                    importantwords.append(str(sent[t-1]))
            if(token.tag_ == 'VB'):
                importantwords.append(str(token))
        t+=1
    
    return query, qvocab, importantwords, expansionwords

## Regulations Retrieval Module

In [102]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = STOPWORDS
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [103]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = stemming(data)
    return data

In [104]:
def tfidfreg(docregs, docinput, category):
    
    if(category == 'regulations'):
        d = [] 
    
        for i in docinput:
            if(i in docs):
                ind = docs.index(i)
                r0 = docregs[ind]
                d.append(r0)
            
        if(len(d)>0):
            docregs = d

    regstring = []
    reverseMap = dict()
    origregstring = [] # flattened docregs
    for ind,i in enumerate(docregs):
        for j in i:
            origregstring.append(j)
            reverseMap[j]=docs[ind]
            data = preprocess(j)
            regstring.append(data)       
    
    
    wordregs = regstring
    
    t=0
    remind = []
    for i in regstring:
        if(i=='\xa0\n' or i == ''):
            remind.append(t)
            regstring.remove(i)
            wordregs.remove(wordregs[t])
        t+=1
        
    origreg = []
    t=0
    while(t<len(origregstring)):
        if(t not in remind):
            origreg.append(origregstring[t])
        t+=1
            
    wordregs = regstring
    
    DF = {}
    wlist = []
    for i in range(len(wordregs)):
        tokens = regstring[i]
        for w in tokens.split():
            try:
                DF[w].add(i)
            except:
                DF[w] = {i}
                wlist.append(w)
                
    for i in DF:
        DF[i] = len(DF[i])
    
    total_vocab = [x for x in DF]
    total_vocab_size = len(DF)

# TFIDF Calculation here
    tf_idf = {}
    N = len(regstring)
    for i in range(N):
        tokens = regstring[i].split()
        counter = Counter(tokens)
        for token in np.unique(tokens):
            if(token in wlist):
                tf = counter[token]/len(tokens)
                df = DF[token]
                idf = np.log((N+1)/(df+1))
                try:
                    tf_idf[token] = tf_idf[token] + tf*idf
                except:
                    tf_idf[token] = tf*idf                
            
    return DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg,reverseMap

In [105]:
def q_vector(qvocab, N, total_vocab, DF):
    Q = np.zeros((len(total_vocab)))
    querytokens = qvocab
    counter = Counter(querytokens)
    words_count = len(querytokens)

    query_weights = {}
    
    for token in np.unique(querytokens):
        if(token in DF):
            tf = counter[token]/words_count
            df = DF[token]
            idf = math.log((N+1)/(df+1))

            try:
                ind = total_vocab.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [106]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [107]:
def cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap):
    d_cosines = []
    
    query_vector = q_vector(qvocab, N, total_vocab, DF)

    t=0
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        t=t+1
        
    out = np.array(d_cosines).argsort()[::-1]
    answers = []
    indexs=[]
    for i in out:
        flag=0
        for j in qvocab:
            if(j in importantwords):
                flag=flag+2
            else:
                flag=flag+1
        if(flag>1):
            answers.append([flag,origreg[i]])

    answers.sort(reverse=True)
    distances=list()
    for ele in answers:
        distances.append(ele[0])
        if ele[1] in reverseMap.keys():
            indexs.append(reverseMap[ele[1]])
        else:
            indexs.append(None)
        
    if(k<len(answers)):
        return answers[0:k],indexs,distances
    else:
        return answers,indexs,distances


In [108]:
def vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg,importantwords,reverseMap):
    D = np.zeros((N, len(total_vocab)))
    for i in tf_idf:
        try:
            ind = total_vocab.index(i[1])
            D[i[0]][ind] = tf_idf[i]
        except:
            pass
    A,indexs,distances = cosine_similarity(k, query, total_vocab, N, qvocab, DF, origreg, D, importantwords,reverseMap)
    
    return A,indexs,distances

In [125]:
def answermod(A, queryinput):
    
    ans = []
    ansreg = []
    
    t=0
    while(t<len(A)):
        A[t][0] = 'title'
        A[t][1] = [A[t][1]]
        t+=1
    t=0
    df = pd.DataFrame(A,columns=['title','paragraphs'])
    
    cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
    cdqa_pipeline.fit_retriever(df=df)
    print("Predicting")
    prediction = cdqa_pipeline.predict(queryinput)
    print("Finished prediction")
    return prediction[0], prediction[2]

In [127]:
def QnAmodel(queryinput, docinput,k,category):
    print('Preprocessing query......')
    qvocab, query = queryvocab(queryinput)
    if(category == 'regulations'):
        rules, vocab, vdef, docinput = regextract(docinput, docs, docregs, mainvocab, vocabdef, queryinput, glossary)
        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('tf-idf calculation in progress.....')
        DF, N, tf_idf, total_vocab, total_vocab_size, wordregs, regstring, origreg,reverseMap = tfidfreg(docregs,docinput,category)
#  Can uncomment below lines to save dump
#         with open(DATA + 'tfidf_regs.pkl','wb') as f:
#             pickle.dump(tf_idf,f)


        print('Extracting relevant answer regulations.......')
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A
    elif(category == 'misc'):
        

        vocab = miscvocab
        N = 565 # number of files in misc. Must change this
        
#         Redo the pickles below such as tfidf and total vocab anol from tfidf function
        with open(DATA + 'misc_tf_idf_new.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'misc_total_vocab_new.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'misc_DF_new.json') as f:
            DF = json.load(f)
        with open(DATA + 'misc_origreg_new.pkl', 'rb') as f:
            origreg = pickle.load(f)
        
        with open(DATA + 'miscfiles_mapper.pkl','rb') as filer:
            reverseMap = pickle.load(filer)

        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
        
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A 

            
    elif(category == 'legal case'):
        rules = ltext
        vocab = casevocab
        N = 1492 # number of files in casefiles,must change
        
        with open(DATA + 'case_tf_idf_new.pkl', 'rb') as f:
            tf_idf = pickle.load(f)
        with open(DATA + 'case_total_vocab_new.pkl', 'rb') as f:
            total_vocab = pickle.load(f)
        with open(DATA + 'case_DF_new.json') as f:
            DF = json.load(f)
        with open(DATA + 'case_origreg_new.pkl', 'rb') as f:
            origreg = pickle.load(f)
            
        with open(DATA + 'casefiles_mapper.pkl','rb') as filer:
            reverseMap = pickle.load(filer)

        with open(DATA + 'casefiles_sentences_new.pkl','rb') as filer:
            case_sentences = pickle.load(filer)

        query, qvocab, importantwords, expansionwords = querypreprocess(query, qvocab, definitions, finaltopics)
        print('Extracting relevant answer regulations.......')
                
        A,indexs,distances = vectormodel(query, k, total_vocab, tf_idf, N, qvocab, DF, origreg, importantwords,reverseMap)
        Aold = A

        Anew = []
        for ans in A:
            ind = origreg.index(ans[1])
            for sent in case_sentences[ind]:
                Anew.append([ans[0],sent])
        A = Anew
        
    else:
        return None
        
    print('Extracting answer')
    answer, ansreg = answermod(A, queryinput)
    return answer, ansreg, Aold,indexs,distances
#     return None, None, Aold,indexs,distances
#     return Aold


## User input

In [133]:
# queryinput = "What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?"
# docinput = 'All'
# category = 'regulations'

queryinput = "I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?"
docinput = 'All'
category = 'legal case'
# queryinput = "How much amount will be provided as reward for informants in insider trading cases"
# docinput = 'All'
# category = 'regulations'
k=15
answer,ansreg,Aold,indexs,distances = QnAmodel(queryinput,docinput,k, category)

Preprocessing query......
Extracting relevant answer regulations.......


/home/spawnfire/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


Extracting answer
Predicting
Finished prediction


In [134]:
print(queryinput)
print()
print(answer)
print()
print(ansreg)

I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?

As per annual returns

 5. As per annual returns filed with MCA by Noticee No. 5, Noticee No. 1 was holding from 25% to 50% of the paid up share capital of Noticee No. 5 during the Investigation Period, the details of which are presented below: Table No. 2: Shareholding of Sanjay Dutt in TCPPL Order in the matter of New Delhi Television Limited – 5 Entities Page 4 of 72 As per the Annual return filed. Shareholding as on September 30, 2006 As per the Annual return filed. Shareholding as on September 29, 2007 As per the Annual return filed. Shareholding as on September 15, 2008 Number of shares % of Total 10000 20000 50 100 Number of shares 5000 20000 % of Total Number of shares % of Total 25 100 6667


In [135]:
Anew=list()
for x in Aold:
    Anew.append(x[1])
Anew = Anew[:k]
docnames=indexs[:k]
dist=distances[:k]
targets = [i for i in range(len(docnames))]

In [136]:
def resultgraph(targets,answers,doclinks,queryinput,distances):
    idoc="results"
    docnumber = 2
#     targets = [6,7,8,9,10]
#     labels = Aold[:5]
#     doclinks = docs[:5]
    
    q = 'Results map '
    edge_data = zip(targets, answers,doclinks,distances)
    
    #nt = Network(height="750px", width="100%", heading=q)
    nt = Network(height="100%", width="100%",heading=q)
    nt.hrepulsion()
        
    nt.add_node("questionDot", label='question', title=queryinput, color = "red")
        
    t=0
    igcount=0
    lccount=0
    for e in edge_data:
        dst = e[0]
        h = e[1]
        pdflink = e[2]
        nt.add_node(str(dst), label=pdflink, title=h, color = "blue", url='./PDFS/'+pdflink+'.pdf')
        nt.add_edge("questionDot", str(dst), value=8, color = "#000000", length=e[3])
        
        t=t+1
    
    nt.show_buttons()  # Allows for adjustments in UI
    nt.show(idoc+".html") # saves it, check for show so that it is automatically opened for user
    
    with open(idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]

    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)

    t=0
    appendind = []
    for i in tokens1:
        if('return' in i):
            appendind.append(t)
        t=t+1
    
    with open("interactive.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<appendind[-1]):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = appendind[-1]
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s

    soup.find_all('script',type="text/javascript")[-1] = tag
    with open(idoc+'.html', "w") as outf:
        outf.write(str(soup))
        
    with open(idoc+'.html') as inf:
        txt = inf.read()
        soup = bs4.BeautifulSoup(txt)  
    c = soup.find_all('script',type="text/javascript")
    tag = c[-1]
    
    t=0
    start = 0
    tindex = []
    tokens1 = tag.contents[0].splitlines(True)
    
    t0=0
    while(t0<len(tokens1)):
        if('"edges": {' in tokens1[t0]):
            break
        t0+=1
        
    with open("nodefont.js") as f:
        jstext = f.read()
    
    p = []
    t=0
    while(t<t0):
        p.append(tokens1[t])
        t=t+1
    p.append(jstext)
    t = t0
    while(t<len(tokens1)):
        p.append(tokens1[t])
        t=t+1
    
    s = ''
    for i in p:
        if(i == '\n'):
            s = s + ' ' + '\n'
        else:
            s = s + ' ' + i
        
    tag.string = s
    
    soup.find_all('script',type="text/javascript")[-1] = tag
    with open(idoc+'.html', "w") as outf:
        outf.write(str(soup))


In [137]:
resultgraph(targets,Anew,docnames,queryinput,dist)

### This is for casefiles

In [91]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):
    output_string = StringIO()
    with open(file_path, 'rb') as in_file:
        parser = PDFParser(in_file)
        doc = PDFDocument(parser)
        rsrcmgr = PDFResourceManager()
        device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.create_pages(doc):
            interpreter.process_page(page)

    return(output_string.getvalue())


def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename


def split_to_title_and_pagenum(table_of_contents_entry): #Can be used to print regulations chapterwise
    title_and_pagenum = table_of_contents_entry.strip()

    title = None
    pagenum = None

    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -2
            while title_and_pagenum[i].isdigit():
                i -= 1

            title = title_and_pagenum[:i].strip()
            pagenum = int(title_and_pagenum[i:].strip())

    return title, pagenum

In [96]:
# 76bb9580b02f83be8fd8f9af3e71f85b.pdf
# WHY IS IT NOT ABLE TO CONVERT THIS DOCUMENT INTO TEXT??????
convert_pdf_to_string("data/informal_guidance/76bb9580b02f83be8fd8f9af3e71f85b.pdf")

'\x0c\x0c\x0c'

In [34]:

from tqdm import tqdm as tqdm
from glob import glob
from os import path

print('started running')

for file in tqdm(glob('All Adjudication PIT orders/*.pdf')):
    file_name = path.basename(file)
    file_str, _ = file_name.split('.')
    if not path.exists(f"./text_casefiles/{file_str}.txt"):
        with open(f"./text_casefiles/{file_str}.txt", "w") as f:
            try:
                f.write(convert_pdf_to_string(file))
            except:
                print("File error :", file_name)
    else:
        "Dates repeat ??"


  0%|          | 0/1492 [00:00<?, ?it/s]

started running



 20%|██        | 300/1492 [46:58<35:41,  1.80s/it]


 40%|████      | 601/1492 [1:01:07<15:27,  1.04s/it]


 60%|█████▉    | 895/1492 [1:11:25<15:33,  1.56s/it]


 76%|███████▌  | 1132/1492 [1:24:39<07:55,  1.32s/it]

File error : 1509102937884.pdf



 95%|█████████▌| 1419/1492 [1:37:32<01:35,  1.30s/it]


100%|██████████| 1492/1492 [1:43:24<00:00,  2.21s/it]

In [16]:
with open(DATA + 'case_origreg.pkl', 'rb') as f:
    origreg = pickle.load(f)

print(type(origreg))
print(type(origreg[0]))
print(len(origreg))


<class 'list'>
<class 'str'>
1496


In [19]:
lsent[0]

[' Adjudication Order in respect of in respect of Zeppelin Investments P Ltd in the matter of Nagpur Power & Industries Limited. BEFORE THE ADJUDICATING OFFICER SECURITIES AND EXCHANGE BOARD OF INDIA [ADJUDICATION ORDER Ref No.: Order AP AS 2019-20 7132] UNDER SECTION 15-I OF SECURITIES AND EXCHANGE BOARD OF INDIA ACT, 1992 READ WITH RULE 5 OF SEBI (PROCEDURE FOR HOLDING INQUIRY AND IMPOSING PENALTIES) RULES, 1995. In respect of: Zeppelin Investment Private Limited (PAN: AAACZ0512H) Nirmal, 20th Floor, Nariman Point, Mumbai-400021. In the matter of Nagpur Power & Industries Limited',
 '1. Nagpur Power & Industries Limited (hereinafter referred to as ‘NPIL’ or ‘Company’) is a listed company having its shares listed on Bombay Stock Exchange Limited (BSE). BSE and National Stock Exchange Limited (NSE) conducted a joint examination regarding the trading activities of certain investors (hereinafter referred to as ‘Group’) in certain scrips during December 15, 2011 to October 09, 2014 (herei

In [73]:
with open(DATA + 'case_origreg_new.pkl', 'rb') as f:
    origreg = pickle.load(f)

print(len(origreg))
# print(len(origreg))
# origreg[0]


1492


In [74]:
import re
# prog = re.compile(" [\d]+\. .*?(?= [\d]+\.)")
cases_sentences = list()

for reg in origreg:
    extracted_points = list()
    onetoNine = re.compile(" \d\. .*?(?= \d+\.)")
    temp = onetoNine.findall(reg)
    extracted_points = extracted_points + temp
    
    prog = re.compile(" \d\d\. .*?(?= \d\d+\.)")
    # after last extracted point in 10 to 100 matches, take the rest of the string from there
    temp = prog.findall(reg)
    try:
        finalpost = reg.find(temp[-1])
        temp.append(reg[finalpost:].replace(temp[-1],""))
        extracted_points = extracted_points+temp
    except:
        ;
    cases_sentences.append(extracted_points)
#     for i in extracted_points:
#         print(i)
#         print("======================")
#     break

In [75]:
len(cases_sentences)

1492

In [76]:
with open(DATA + 'casefiles_sentences_new.pkl','wb') as filer:
    pickle.dump(cases_sentences,filer)


In [13]:
from os import listdir
from os.path import isfile, join
mypath = "text_casefiles/"
case_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(case_files)

1492

In [66]:
# case_files_mapper = dict()
# casefiles_origreg_new=list()
# for f in case_files:
#     with open('text_casefiles/'+f, 'r') as filer:
#         temp = filer.read()    
#     case_files_mapper[' '.join(temp.split())]=f.split('.txt')[0]
#     casefiles_origreg_new.append(' '.join(temp.split()))


In [44]:
case_files_mapper.values()

dict_values(['1522152063550', '1533038755477', '1516011484965', '1400572608980', '1516120058333', '1529063091218', '1413277148190', '1316430610006', '1395314338991', '1400843954687', '1520251280149', '1287986788991', '1348722632964', '1561556788672', '1539610420169', '1519385836881', '1404722657323', '1451304247597', '1539244669020', '1554987857746', '1561373468875', '1522294817162', '1446202343984', '1404297629677', '1288602960748', '1519367946045', '1368097637407', '1562557977201', '1540984062249', '1601472004286_2', '1323316808783', '1430391005051', '1427797836152', '1604401204480_3', '1511178146002', '1497618427672', '1404800119683', '1545202196705', '1558707076654', '1404464457988', '1375097182210', '1409139682301', '1328781095177', '1343802475735', '1603367409459_1', '1509195779917', '1569832555505', '1524802106584', '1497435799136', '1385639782643', '1419586137409', '1556262533151', '1387262118831', '1509361107150', '1424941694848', '1513938583180', '1407907121839', '15613896266

In [45]:
with open(DATA + 'case_origreg_new.pkl','wb') as filer:
    pickle.dump(casefiles_origreg_new,filer)


In [46]:
with open(DATA + 'casefiles_mapper.pkl','wb') as filer:
    pickle.dump(case_files_mapper,filer)


In [28]:
# print(type(origreg))
# print(type(origreg[0]))
# print(origreg[0])
casefiles_origreg_new

NameError: name 'casefiles_origreg_new' is not defined

In [30]:
regstring = []
origregstring = [] # flattened docregs
for ind,j in enumerate(origreg):
    origregstring.append(j)
    data = preprocess(j)
    regstring.append(data)       


wordregs = regstring

t=0
remind = []
for i in regstring:
    if(i=='\xa0\n' or i == ''):
        remind.append(t)
        regstring.remove(i)
        wordregs.remove(wordregs[t])
    t+=1

origreg = []
t=0
while(t<len(origregstring)):
    if(t not in remind):
        origreg.append(origregstring[t])
    t+=1

wordregs = regstring

DF = {}
wlist = []
for i in range(len(wordregs)):
    tokens = regstring[i]
    for w in tokens.split():
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
            wlist.append(w)

for i in DF:
    DF[i] = len(DF[i])

total_vocab = [x for x in DF]

# TFIDF Calculation here
tf_idf = {}
N = len(regstring)
for i in range(N):
    tokens = regstring[i].split()
    counter = Counter(tokens)
    for token in np.unique(tokens):
        if(token in wlist):
            tf = counter[token]/len(tokens)
            df = DF[token]
            idf = np.log((N+1)/(df+1))
            try:
                tf_idf[token] = tf_idf[token] + tf*idf
            except:
                tf_idf[token] = tf*idf                


In [32]:
with open(DATA + 'case_tf_idf_new.pkl', 'wb') as f:
    pickle.dump(tf_idf,f)
with open(DATA + 'case_total_vocab_new.pkl', 'wb') as f:
    pickle.dump(total_vocab,f)
with open(DATA + 'case_DF_new.json',"w") as f:
    json.dump(DF, f, indent = 6) 


In [27]:
type(docregs[0])

list

In [30]:
len(casefiles)

1492

In [25]:
# case_files_map.keys().

### For misc

In [18]:
with open(DATA + 'misc_origreg.pkl', 'rb') as f:
    origreg = pickle.load(f)

print(type(origreg))
print(type(origreg[0]))
print(len(origreg))


<class 'list'>
<class 'str'>
447


In [119]:
with open(DATA + 'misc_origreg_new.pkl', 'rb') as f:
    origreg = pickle.load(f)

print(type(origreg))
# print(origreg[:3])
print(len(origreg))
# print()
origreg = list(filter(None, origreg))
print(len(origreg))

<class 'list'>
202
202


In [123]:
with open(DATA + 'miscfiles_mapper.pkl','rb') as filer:
    reverseMap = pickle.load(filer)
print(reverseMap[origreg[0]])
origreg[0]

1593416996354


'WORKING GROUP REPORT ON SOCIAL STOCK EXCHANGE 1 Chairman’s Note The Hon’ble Finance Minister’s bold vision of a Social Stock Exchange could not have come at a more opportune time for India. In a country whose policymakers have always striven to create economic prosperity for all, the seeds of a form of capital that would prioritize social returns over financial returns have always been present in the policymaker’s conception of Indian business and enterprise. It is no wonder then that India is the only country in the world to have mandated a Corporate Social Responsibility role for its private enterprises, and that it is also one of the world’s fastest growing impact investing destinations. Yet, much more can and should be done. India’s economic imperative is to feed, clothe, educate and empower more than a billion people, in ways that conserve and grow its natural, cultural and social heritages. It cannot expect to accomplish this lofty objective on the strength of conventional comme

In [120]:
import re
# prog = re.compile(" [\d]+\. .*?(?= [\d]+\.)")
misc_sentences = list()

for reg in origreg:
    extracted_points = list()
    onetoNine = re.compile(" \d\. .*?(?= \d+\.)")
    temp = onetoNine.findall(reg)
    extracted_points = extracted_points + temp
    
    prog = re.compile(" \d\d\. .*?(?= \d\d+\.)")
    # after last extracted point in 10 to 100 matches, take the rest of the string from there
    temp = prog.findall(reg)
    try:
        finalpost = reg.find(temp[-1])
        temp.append(reg[finalpost:].replace(temp[-1],""))
        extracted_points = extracted_points+temp
    except:
        ;
    misc_sentences.append(extracted_points)
    for i in extracted_points:
        print(i)
        print("======================")
    break

 5. This creates an artificial distinction between types of CSR expenditure, with some qualifying for tax exemption while others do not.  Remove the 10% cap on income eligible for deduction under 80G, for donations to all NPOs that benefit from the SSE. In conversations with stakeholders, this restriction came up as a significant barrier towards greater philanthropic giving.  Allow first time retail investors (who are investing in the SSE for the first time) to avail a 100% tax exemption on their investments in the SSE MF structure, subject to an overall limit of INR 1 Lakh. There is a precedent for this through the 80CCG section of the IT Act. NPOs  Enable fast-tracking of getting certifications for 12A, 12AA and 80G for all NPOs doing social and financial reporting, as per the guidelines outlined in chapter
 3. Note: In time, the minimum reporting standard, as it becomes more rigorous and sophisticated, should entirely replace the 12A, 12AA, 80G certifications process.  Re-evalua

In [58]:
from os import listdir
from os.path import isfile, join
mypath = "text_misc_clean/"
misc_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
len(misc_files)

565

In [59]:
misc_files_mapper = dict()
miscfiles_origreg_new=list()

for f in misc_files:
    with open('text_misc_clean/'+f, 'r') as filer:
        temp = filer.read()    
    misc_files_mapper[' '.join(temp.split())]=f.split('.txt')[0]
    miscfiles_origreg_new.append(' '.join(temp.split()))

In [118]:
# with open(DATA + 'misc_origreg_new.pkl','wb') as filer:
#     pickle.dump(origreg,filer)

# with open(DATA + 'miscfiles_mapper.pkl','wb') as filer:
#     pickle.dump(reverseMap,filer)


In [110]:
regstring = []
origregstring = [] # flattened docregs
for ind,j in enumerate(origreg):
    origregstring.append(j)
    data = preprocess(j)
    regstring.append(data)       


wordregs = regstring

t=0
remind = []
for i in regstring:
    if(i=='\xa0\n' or i == ''):
        remind.append(t)
        regstring.remove(i)
        wordregs.remove(wordregs[t])
    t+=1

origreg = []
t=0
while(t<len(origregstring)):
    if(t not in remind):
        origreg.append(origregstring[t])
    t+=1

wordregs = regstring

DF = {}
wlist = []
for i in range(len(wordregs)):
    tokens = regstring[i]
    for w in tokens.split():
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
            wlist.append(w)

for i in DF:
    DF[i] = len(DF[i])

total_vocab = [x for x in DF]

# TFIDF Calculation here
tf_idf = {}
N = len(regstring)
for i in range(N):
    tokens = regstring[i].split()
    counter = Counter(tokens)
    for token in np.unique(tokens):
        if(token in wlist):
            tf = counter[token]/len(tokens)
            df = DF[token]
            idf = np.log((N+1)/(df+1))
            try:
                tf_idf[token] = tf_idf[token] + tf*idf
            except:
                tf_idf[token] = tf*idf                


In [111]:
with open(DATA + 'misc_tf_idf_new.pkl', 'wb') as f:
    pickle.dump(tf_idf,f)
with open(DATA + 'misc_total_vocab_new.pkl', 'wb') as f:
    pickle.dump(total_vocab,f)
with open(DATA + 'misc_DF_new.json',"w") as f:
    json.dump(DF, f, indent = 6) 


In [33]:
len(informalnames)+len(conceptnames)

447

In [35]:
temp = informalnames+conceptnames
for i in temp:
    if i not in misc_files:
        print(i)
print("=============")

for i in misc_files:
    if i not in temp:
        print(i)


555d1756aaeb1f62fd2c88e898a2f555.txt
23081ce213a6d025dc456172ad94076e.txt
22.txt
1293522755063.txt
854.txt
1290762059866.txt
835.txt
67.txt
671.txt
737.txt
1290762190460.txt
20.txt
84.txt
b52c31e1128d9a5b3bf0a100991c043c.txt
b072ee9db424478ed3530dbf2011de06.txt
b604b3377f1b59fe672cb19703a846be.txt
31e3796ec68c154f5064c89fea250100.txt
1593416996354.txt
1289624747835.txt
1577111310240.txt
74c8cd5e7b8eaff5a32e6f5f607b091c.txt
cb6c47c0382e129a77a82b7aeef2af8c.txt
1605853267317.txt
1606126221923.txt
43c5265fb89ee26b1fad81d16724ad68.txt
bf78f61b3413010ec1bb7e165e3f02e8.txt
1441101030969.txt
81b4dc17c9f1be350e3b47061c57a392.txt
1525440570949.txt
b59a62c0fe1695d072aae10e6c761ec3.txt
b764b6ad85c8fd138a12e5acf465cad8.txt
1600665983575.txt
1534132138682.txt
1532603980878.txt
1543549361996.txt
ba103048b995729b77151a88ac672854.txt
1547636489300.txt
b2e3ef598ab30e0130dccca7bb5c3115.txt
b32ca837ca65e79828dddb2c7936f4cf.txt
ca8c8a953c61395416ca299bf5a818ec.txt
1558349599905.txt
1558352332468.txt
15857

In [46]:
miscfiles = dict()
for filename in misc_files:
    with open('text_misc_clean/'+filename, 'r') as f:
        temp = f.read()
    miscfiles[filename]=' '.join(temp.split())

In [17]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?


chapter IV


(2) The listed entity which has listed its specified securities shall comply with the corporate governance provisions as specified in chapter IV which shall be implemented in a manner so as to achieve the objectives of the principles as mentioned below. (a) The rights of shareholders: The listed entity shall seek to protect and facilitate the exercise of the following rights of shareholders: (i)  right to participate in, and to be sufficiently informed of, decisions concerning fundamental corporate changes. (ii) opportunity to participate effectively and vote in general shareholder meetings. (iii)being  informed  of  the  rules,  including  voting  procedures  that  govern  general shareholder meetings. (iv) opportunity  to  ask  questions  to  the  board  of  directors,  to  place  items  on  the agenda of general meetings, and to propose resolutions, subject to reas

In [18]:
queryinput = "As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?"              
docinput = 'All'
category = 'misc'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,10, category)

Preprocessing query......
Extracting relevant answer regulations.......
Extracting answer


In [19]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?


the maximum number of public companies in which a person can be appointed as a director shall not exceed ten


twenty companies at the same time: Provided that the maximum number of public companies in which a person can be appointed as a director shall not exceed ten.


In [20]:
queryinput = "What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
docinput = 'All'
category = 'misc'
answer, ansreg, Aold = QnAmodel(queryinput,docinput,10, category)

Preprocessing query......
Extracting relevant answer regulations.......
Extracting answer


In [21]:
print(queryinput)
print('\n')
print(answer)
print('\n')
print(ansreg)


What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?


integrity, experience and solvency requirements


facilitate fund raising by fraudulent entities. It is therefore important to specify integrity, experience and solvency requirements applicable to crowdfunding platform owner and the key persons associated with it. Therefore it is proposed that any online offering or issue or sale through the internet can be made only through a SEBI recognized crowdfunding platform.


In [22]:
for a in Aold:
    print(a[1])
    print("\n======\n")

Discussion Paper on1. Alternate Capital Raising Platform and2. Review of other regulatory requirements1. Background1.1. With a view to provide easier exit options for informed investors like Angel Investors, Venture Capital Funds (VCFs), etc, SEBI Board in its meeting held on June 25, 2013, approved a proposal to permit listing of SMEs in Institutional Trading platform (ITP) without having to make an IPO.1.2. While such companies are listed on the ITP, they are not permitted to raise equity capital through public issues though they can continue to make private placements. SEBI has been in receipt of suggestions to permit capital raising in the said ITP platform.1.3. Further, as part of the regular consultative process, SEBI in the recent past interacted with iSPIRT (which stands for 'Indian Software Product Industry Round Table', a body dedicated to promote existing Indian software product companies). A few Indian start-ups were also present during the said interaction. It was highligh

In [23]:
def get_res(query, docinput, category, qn_ind):
    
    A = QnAmodel(query, docinput, 10, category)

    out_file = "temp_results_annotated_tfidf/" + qn_ind + ".txt"
    out_fptr = open(out_file, 'w+')
    print("Query: ", query, file=out_fptr)
    print("docinput: ", docinput, file=out_fptr)
    print("category: ", category, file=out_fptr)
    
    print("Printing top 10 results from retreiver model:", file=out_fptr)
    for num_id, i in enumerate(A[:10]):
        print("Result ", num_id, ":", file=out_fptr)
        print(i, file=out_fptr)
        print("=====", file=out_fptr)
    out_fptr.close()

In [ ]:
queries_case = ["I am a shareholder in xyz company with my total share capital being 7%. I sold a part of my share percent which reduced my holding by 3%. Will I be required to disclose about the same to under Form C to the company since my total shareholding has fallen below 5%?"
               ,"I am a shareholder in abc company with a shareholding of 10%. I recently acquired a few more shares which has increased my shareholding by 5%, bringing it to a total of 15%. What all disclosures do I need to make?"
               ,"PQR, a corporate entity has purchased 8986 shares and sold 928600 shares of ASQ.  PQR also purchased 9,86,094 shares from off-market. As of 31st December, 2004, PQR was holding 9,86,094 shares which was which was 9.26% of the share capital of ASQ. By March 31st, its shareholding was reduced to 0. What all violations have been made?"
               ,"GHI is a corporate entity that is a promoter of SPZ. As on March 2005, it held 2,30,59,317 shares constituting of 30.87% of the total shares. By the end of September 2005, its shareholding reduced by 1,41,92,187 to 19% . What all violations have been made?"
               ,"TPR is a company that is listed on both BSE & NSE. STC is a promoter of TPR. TPR took out Rights Issue and through that allotted 16,49,653 partly paid-up shares to STC on 2nd October, 2012. Consequently, the total shares of STC  increased from 1,03,18,427 constituting to 4.98% of the share capital to 1,19,68,080 shares constituting 5.06% of the total share capital of TPR. STC was informed about the allotment on 10th October, 2012. However, STC did not make disclosures about its increased shareholding under 13(1) of PIT Regulations and believed that since it was already a promoter of TPR, it instead made disclosure under 14(4A) of PIT Regulations. Accordingly it made disclosure under 13(4A) of PIT Regulations under Form D with a delay of 1 month and not under 13(1) of PIT Regulations under Form A. Is there any violation?"
               ,"Mr. X is a promoter and managing director of GTF Pharmaceuticals. He has pledged certain shares of the company and the pledge on these shares was subsequently released. Mr. X had created a pledge 1,20,000 shares on 11.12.2012 and release of pledge of 5,00,000 shares had taken place on 27.12.2012. Later it was observed that shareholding of Mr. X was changed after some off-market transactions. Mr. X has contended that 1,20,000 shares were pledged with YZ Financial on 11.12.2012 and the said transaction was intimated to stock exchanges on 21.12.2012 and there was a delay of 3 days and the same may be condoned. On 27.12.2012 the pledge on 5 lakh shares were released by DTR and the same was duly intimated to stock exchanges on 04.01.2013. With respect to the off-market transaction, Mr. X has contended that the same was No an off-market transaction but was only a transfer from the his one DP account to its another DP account. It has been alleged that Mr. X did not make the requisite disclosures under Regulation 31(1) &31(2) read with 31(3) of the SAST Regulations, 2011 and Regulation 13(4) & (4A) read with 13(5) SEBI (Prohibition of Insider Trading) Regulations,1992 (PIT Regulations). Has the violation occurred?"
               ,"Mr. X is a promoter and managing director of GTF Pharmaceuticals. He has pledged certain shares of the company and the pledge on these shares was subsequently released. Mr. X had created a pledge 1,20,000 shares on 11.12.2012 and release of pledge of 5,00,000 shares had taken place on 27.12.2012. Later it was observed that shareholding of Mr. X was changed after some off-market transactions. Mr. X has contended that 1,20,000 shares were pledged with YZ Financial on 11.12.2012 and the said transaction was intimated to stock exchanges on 21.12.2012 and there was a delay of 3 days and the same may be condoned. On 27.12.2012 the pledge on 5 lakh shares were released by DTR and the same was duly intimated to stock exchanges on 04.01.2013. With respect to the off-market transaction, Mr. X has contended that the same was No an off-market transaction but was only a transfer from the his one DP account to its another DP account. It has been alleged that Mr. X did not make the requisite disclosures under Regulation 31(1) &31(2) read with 31(3) of the SAST Regulations, 2011 and Regulation 13(4) & (4A) read with 13(5) SEBI (Prohibition of Insider Trading) Regulations,1992 (PIT Regulations). How much penalty should be imposed?"]

In [ ]:
for ind, queryinput in enumerate(queries_case):
    get_res(queryinput, 'All', 'legal case', "CaseFile_"+str(ind))

In [ ]:
queries_new_micro = ["What are some of the activities that being outsourced by intermediaries in the securities market?"
                    ,"Under what conditions can approval from shareholders be dispensed with for reclassification of promoter to public?"
                    ,"What is transmission of shares?"
                    ,"What are the provisions for transfer of shares?"]

In [ ]:
for ind, queryinput in enumerate(queries_new_micro):
    get_res(queryinput, 'All', 'misc', "New_Microsoft_doc_"+str(ind))

In [ ]:
queries_pit = ["Is it insider trading if I know an employee of a company and I buy that company’s stock?"
              ,"How much amount will be provided as reward for informants in insider trading cases by Securities and Exchange Board of India (SEBI)?"
              ,"How do CEOs and their families trade stock from their company without it being insider trading?"
              ,"When do CEOs and their families trade stock from their company without it being insider trading?"
              ,"What should be done by a company when it suspects insider trading by an employee?"
              ,"Who is a compliance officer and what is its role in preventing Insider trading?"]

In [ ]:
for ind, queryinput in enumerate(queries_pit):
    get_res(queryinput, 'All', 'regulations', "PIT_"+str(ind))

In [ ]:
queries_query_testing = ["What if a stock delisted from the market?"
                        ,"What can a person do with shares that are now delisted?"
                        ,"Why companies are delisting their shares?"
                        ,"If a company is about to delist and is ready to buy back my shares and I didn't sell, what will happen?"
                        ,"How does an initial public offering get valued?"
                        ,"How does a company offer initial public offering?"
                        ]

In [ ]:
for ind, queryinput in enumerate(queries_query_testing):
    get_res(queryinput, 'All', 'regulations', "query_testing_"+str(ind))

In [ ]:
queries_sast = ["What are the threshold limits for acquisition of shares / voting rights, beyond which an obligation to   make an open offer is triggered?"
               ,"What is a voluntary open offer?"
               ,"What are the restrictions on acquirers making a voluntary open offer?"]

In [ ]:
for ind, queryinput in enumerate(queries_sast):
    get_res(queryinput, 'All', 'regulations', "sast_"+str(ind))

In [ ]:
queries_sebi_consult = ["What are the legal  provisions  on  appointment  and  re-appointment  of directors  of  a listed entity?"
                       ,"As of date, what are the provisions for a person to get appointed of persons who fail to get elected as Whole time Directors /Managing Directors at the general meeting of a listed entity?"
                       ,"What are the key requirements for issuer companies to list under Innovators Growth Platform (IGP)?"
                       ,"What is meant by 'Migration to Main Board'?"
                       ,"Why are Broad Market Indices created?"
                       ,"Which are the major Domestic & International broad market indices?"]

In [ ]:
for ind, queryinput in enumerate(queries_sebi_consult):
    get_res(queryinput, 'All', 'misc', "sebi_consult_"+str(ind))